<a href="https://colab.research.google.com/github/Lanisha20/ECEN-360/blob/main/FHTH_v1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# From Hashtags to Hangars: Fashion Sales Forecast Model

# INSTALLS (Uncomment if running for first time)
#!pip install xgboost textblob seaborn matplotlib snscrape pytrends prophet streamlit
# allows for multiple files & multi-trend input

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from pytrends.request import TrendReq
from prophet import Prophet
import streamlit as st
import time
import random

In [ ]:
# ----------------------------------------
# MULTI-FILE UPLOAD + COMPARISON
# ----------------------------------------
def load_multiple_csv(files):
    dfs = []
    for file in files:
        df = load_and_clean_csv(file)
        dfs.append(df)
    return dfs

def compare_multiple_datasets(dfs, labels):
    st.subheader("Comparison of Multiple Sales Datasets")
    combined_data = []

    for df, label in zip(dfs, labels):
        monthly_sales = df.groupby('Month')['Purchase Amount (USD)'].sum().reset_index()
        monthly_sales['Dataset'] = label
        combined_data.append(monthly_sales)

    combined_df = pd.concat(combined_data)

    fig, ax = plt.subplots(figsize=(12,6))
    sns.lineplot(data=combined_df, x='Month', y='Purchase Amount (USD)', hue='Dataset', marker='o')
    plt.title("Monthly Sales Comparison")
    st.pyplot(fig)

# ----------------------------------------
# STEP 1: Load + Clean the CSV (Updated to include Price and Review Rating)
# ----------------------------------------
def load_and_clean_csv(filepath):
    df = pd.read_csv(filepath)
    df['Date Purchase'] = pd.to_datetime(df['Date Purchase'], errors='coerce')
    df = df.dropna(subset=['Date Purchase', 'Purchase Amount (USD)', 'Item Purchased'])
    if 'Price' not in df.columns:
        df['Price'] = np.random.uniform(10, 500, size=len(df))  # Mock price if not present
    if 'Review Rating' not in df.columns:
        df['Review Rating'] = np.random.uniform(1, 5, size=len(df))  # Mock ratings if not present
    df['Month'] = df['Date Purchase'].dt.month
    df['Week'] = df['Date Purchase'].dt.isocalendar().week.astype(int)
    df['Item_Code'] = df['Item Purchased'].astype('category').cat.codes
    return df

# ----------------------------------------
# STEP 2: Visualize Data
# ----------------------------------------
def visualize_sales(df):
    st.subheader("Total Sales per Month")
    monthly_sales = df.groupby('Month')['Purchase Amount (USD)'].sum().reset_index()
    fig, ax = plt.subplots(figsize=(10,5))
    sns.barplot(data=monthly_sales, x='Month', y='Purchase Amount (USD)', ci=None)
    st.pyplot(fig)

    st.subheader("Sales Distribution for Top Items")
    top_items = df['Item Purchased'].value_counts().head(10).index
    fig, ax = plt.subplots(figsize=(10,5))
    sns.boxplot(data=df[df['Item Purchased'].isin(top_items)], x='Item Purchased', y='Purchase Amount (USD)')
    plt.xticks(rotation=45)
    st.pyplot(fig)

# ----------------------------------------
# UPDATED INDIVIDUAL DATASET ANALYTICS WITH BUTTONS AND SIDE-BY-SIDE LAYOUT
# ----------------------------------------

def individual_dataset_analytics(datasets, labels):
    st.header("Individual Dataset Analytics")
    if "show_analytics" not in st.session_state:
        st.session_state["show_analytics"] = False

    if st.button("Show Analytics for All Datasets") or st.session_state["show_analytics"]:
        st.session_state["show_analytics"] = True

        cols = st.columns(len(datasets))
        for col, df, label in zip(cols, datasets, labels):
            with col:
                st.subheader(f"Analytics for {label}")
                visualize_sales(df)
                model = train_sales_predictors(df)

                trends_input = st.text_input(f"Enter trends to forecast for {label} (comma-separated):", key=f"forecast_{label}")
                if trends_input:
                    trend_list = [trend.strip() for trend in trends_input.split(",")]
                    forecast_multiple_trends(trend_list)

                forecast_months = st.slider(f"Select forecast months for {label}", min_value=3, max_value=24, value=6, key=f"months_{label}")
                forecast_total_sales(df, periods=forecast_months)

                predict_top_items(df, model)

                st.markdown("---")

# ----------------------------------------
# STEP 3: Train Sales Predictors (Upgraded with XGBoost and Random Forest)
# ----------------------------------------
def train_sales_predictors(df):
    features = ['Month', 'Week', 'Item_Code', 'Price', 'Review Rating']
    X = df[features]
    y = df['Purchase Amount (USD)']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    xgb_model = XGBRegressor(n_estimators=50, learning_rate=0.1, random_state=42)
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    xgb_model.fit(X_train, y_train)
    rf_model.fit(X_train, y_train)

    xgb_preds = xgb_model.predict(X_test)
    rf_preds = rf_model.predict(X_test)

    xgb_mae = mean_absolute_error(y_test, xgb_preds)
    xgb_r2 = r2_score(y_test, xgb_preds)

    rf_mae = mean_absolute_error(y_test, rf_preds)
    rf_r2 = r2_score(y_test, rf_preds)

    st.subheader("Model Performance")
    st.write(f"XGBoost MAE: {xgb_mae:.2f}, R²: {xgb_r2:.2f}")
    st.write(f"Random Forest MAE: {rf_mae:.2f}, R²: {rf_r2:.2f}")

    st.subheader("Feature Importance (XGBoost)")
    importance = xgb_model.feature_importances_
    importance_df = pd.DataFrame({'Feature': features, 'Importance': importance}).sort_values(by='Importance', ascending=False)

    fig, ax = plt.subplots(figsize=(10,5))
    sns.barplot(data=importance_df, x='Feature', y='Importance')
    plt.title('Feature Importance')
    st.pyplot(fig)

    return xgb_model

# # ----------------------------------------
# # MULTI-TREND FORECASTING FUNCTION
# # ----------------------------------------
# def forecast_multiple_trends(keywords, days=7):
#     pytrends = TrendReq(hl='en-US', tz=360)
#     fig, ax = plt.subplots(figsize=(12,6))

#     for keyword in keywords:
#         pytrends.build_payload([keyword], cat=0, timeframe='today 3-m')
#         df = pytrends.interest_over_time()

#         if not df.empty:
#             df = df.drop(columns='isPartial').reset_index().rename(columns={keyword: 'y', 'date': 'ds'})
#             model = Prophet()
#             model.fit(df)
#             future = model.make_future_dataframe(periods=days)
#             forecast = model.predict(future)
#             ax.plot(forecast['ds'], forecast['yhat'], label=keyword)

#     ax.set_title("Trend Forecasts")
#     ax.set_xlabel("Date")
#     ax.set_ylabel("Trend Popularity")
#     ax.legend()
#     st.pyplot(fig)

# ----------------------------------------
# UPDATED FUNCTION: FORECAST MULTIPLE TRENDS (SIMULATED)
# ----------------------------------------

def forecast_multiple_trends(keywords, days=90):
    st.subheader("Forecasted Trends (Simulated)")

    fig, ax = plt.subplots(figsize=(12,6))

    for keyword in keywords:
        # Simulate a realistic trend curve
        dates = pd.date_range(start=pd.Timestamp.now() - pd.Timedelta(days=90), periods=90)
        scores = np.clip(np.random.normal(loc=50, scale=15, size=90), 10, 100)

        trend_df = pd.DataFrame({
            'ds': dates,
            'y': scores
        })

        model = Prophet()
        model.fit(trend_df)
        future = model.make_future_dataframe(periods=days)
        forecast = model.predict(future)
        ax.plot(forecast['ds'], forecast['yhat'], label=keyword)

    ax.set_title("Trend Forecasts (Simulated)")
    ax.set_xlabel("Date")
    ax.set_ylabel("Trend Popularity")
    ax.legend()
    st.pyplot(fig)

# ----------------------------------------
# (Other Functions: forecast_total_sales, predict_top_items remain the same)
# ----------------------------------------
# ----------------------------------------
# FORECAST TOTAL FUTURE SALES
# ----------------------------------------
def forecast_total_sales(df, periods=6):
    sales_df = df[['Date Purchase', 'Purchase Amount (USD)']].rename(columns={
        'Date Purchase': 'ds',
        'Purchase Amount (USD)': 'y'
    })
    sales_df = sales_df.groupby('ds').sum().reset_index()

    model = Prophet()
    model.fit(sales_df)
    future = model.make_future_dataframe(periods=periods, freq='M')
    forecast = model.predict(future)

    st.subheader("Forecasted Total Sales")
    fig2 = model.plot(forecast)
    st.pyplot(fig2)

    return forecast[['ds', 'yhat']]

# ----------------------------------------
# PREDICT TOP SELLING ITEMS
# ----------------------------------------
def predict_top_items(df, model):
    latest_month = df['Month'].max()
    predict_df = df[df['Month'] == latest_month][['Month', 'Week', 'Item_Code', 'Price', 'Review Rating']]
    predict_df['Month'] = (predict_df['Month'] % 12) + 1  # simulate next month

    preds = model.predict(predict_df)
    predict_df['Predicted_Sales'] = preds
    predict_df['Item Purchased'] = df[df['Month'] == latest_month]['Item Purchased'].values

    top_items = predict_df.groupby('Item Purchased')['Predicted_Sales'].sum().sort_values(ascending=False).head(10)

    st.subheader("Top 10 Predicted Best-Selling Items (Next Month)")
    fig, ax = plt.subplots(figsize=(10,5))
    sns.barplot(x=top_items.index, y=top_items.values)
    plt.xticks(rotation=45)
    st.pyplot(fig)

# ----------------------------------------
# UPDATED FUNCTION: CORRELATE TRENDS WITH DATASETS (SIMULATED TREND SCORES)
# ----------------------------------------

def correlate_trends_with_datasets(datasets, labels, keywords):
    st.subheader("Correlate Trends with Sales by Dataset (Simulated)")

    correlation_results = []

    for keyword in keywords:
        trend_monthly = pd.DataFrame({
            'Month': list(range(1, 13)),
            keyword: [random.uniform(30, 80) for _ in range(12)]
        })

        for df, label in zip(datasets, labels):
            sales_monthly = df.groupby('Month')['Purchase Amount (USD)'].sum().reset_index()
            merged = pd.merge(trend_monthly, sales_monthly, on='Month', how='inner')
            if len(merged) > 1:
                corr = merged[keyword].corr(merged['Purchase Amount (USD)'])
                correlation_results.append({"Dataset": label, "Trend": keyword, "Correlation": corr})

    if correlation_results:
        corr_df = pd.DataFrame(correlation_results)
        st.dataframe(corr_df)
    else:
        st.warning("No trend correlation results available.")

# ----------------------------------------
# UPDATED FUNCTION: TREND IMPACT MODELING (SIMULATED)
# ----------------------------------------

def trend_impact_modeling(datasets, labels, keywords):
    st.subheader("Trend Impact Modeling on Sales (Simulated)")

    impact_results = []

    for keyword in keywords:
        trend_monthly = pd.DataFrame({
            'Month': list(range(1, 13)),
            keyword: [random.uniform(30, 80) for _ in range(12)]
        })

        for df, label in zip(datasets, labels):
            sales_monthly = df.groupby('Month')['Purchase Amount (USD)'].sum().reset_index()
            merged = pd.merge(trend_monthly, sales_monthly, on='Month', how='inner')
            if len(merged) > 1:
                X = merged[[keyword]].values
                y = merged['Purchase Amount (USD)'].values
                model = LinearRegression()
                model.fit(X, y)
                coef = model.coef_[0]
                impact_results.append({"Dataset": label, "Trend": keyword, "Impact Coefficient": coef})

    if impact_results:
        impact_df = pd.DataFrame(impact_results)
        st.dataframe(impact_df)
    else:
        st.warning("No impact modeling results available.")

# ----------------------------------------
# MASTER FORECASTING AND BEST DATASET DETECTION SECTION
# ----------------------------------------

def detect_best_dataset(datasets, labels, trends):
    st.subheader("Auto-Detect Best Dataset by Trend Correlation")
    pytrends = TrendReq(hl='en-US', tz=360)
    best_dataset = None
    best_score = -2

    for df, label in zip(datasets, labels):
        total_corr = 0
        count = 0

        for trend in trends:
            try:
                pytrends.build_payload([trend], cat=0, timeframe='today 3-m')
                trend_df = pytrends.interest_over_time()
                if not trend_df.empty:
                    trend_df = trend_df.drop(columns='isPartial').reset_index()
                    trend_df['Month'] = trend_df['date'].dt.month
                    trend_monthly = trend_df.groupby('Month')[trend].mean().reset_index()

                    sales_monthly = df.groupby('Month')['Purchase Amount (USD)'].sum().reset_index()
                    merged = pd.merge(trend_monthly, sales_monthly, on='Month', how='inner')
                    if len(merged) > 1:
                        corr = merged[trend].corr(merged['Purchase Amount (USD)'])
                        if not pd.isna(corr):
                            total_corr += corr
                            count += 1
            except:
                pass

        avg_corr = total_corr / count if count > 0 else -2
        if avg_corr > best_score:
            best_score = avg_corr
            best_dataset = label

    if best_dataset:
        st.success(f"The dataset most aligned with fashion trends is: **{best_dataset}** (Avg Corr: {best_score:.2f})")
    else:
        st.warning("No strong correlations detected.")


def master_combined_forecast(datasets, labels):
    st.subheader("Master Combined Forecast Across Datasets")
    combined_df = pd.concat(datasets)
    combined_df = combined_df[['Date Purchase', 'Purchase Amount (USD)']].rename(columns={
        'Date Purchase': 'ds',
        'Purchase Amount (USD)': 'y'
    })
    combined_df = combined_df.groupby('ds').sum().reset_index()

    model = Prophet()
    model.fit(combined_df)
    future = model.make_future_dataframe(periods=12, freq='M')
    forecast = model.predict(future)

    fig, ax = plt.subplots(figsize=(12,6))
    model.plot(forecast, ax=ax)
    plt.title("Master Combined Sales Forecast")
    st.pyplot(fig)

def scrape_top_fashion_trends():
    st.subheader("Live Google Trends Scraper for Fashion Trends")
    pytrends = TrendReq(hl='en-US', tz=360)
    try:
        trending_searches_df = pytrends.trending_searches(pn='united_states')
        fashion_keywords = [trend for trend in trending_searches_df[0].tolist() if any(fashion_word in trend.lower() for fashion_word in ['fashion', 'outfit', 'style', 'look', 'wear', 'clothing', 'apparel'])]

        if fashion_keywords:
            st.success(f"🧵 Detected {len(fashion_keywords)} fashion-related trending keywords!")
            st.write(fashion_keywords)
            return fashion_keywords
        else:
            st.warning("No fashion-related trends detected right now.")
            return []
    except Exception as e:
        st.error(f"Error scraping Google Trends: {e}")
        return []

# ----------------------------------------
# UPDATED FUNCTION: SENTIMENT ANALYSIS ON TRENDS (SIMULATED)
# ----------------------------------------

def sentiment_analysis_on_trends(trends):
    st.subheader("Sentiment Analysis on Trends (Simulated)")

    sentiments = {}
    for trend in trends:
        roll = random.random()
        if roll < 0.6:
            sentiment = random.uniform(0.2, 1.0)  # positive
        elif roll < 0.9:
            sentiment = random.uniform(-0.2, 0.2)  # neutral
        else:
            sentiment = random.uniform(-1.0, -0.2)  # negative
        sentiments[trend] = sentiment

    sentiment_df = pd.DataFrame({
        'Trend': list(sentiments.keys()),
        'Sentiment Score': list(sentiments.values())
    })

    st.dataframe(sentiment_df)

    return sentiments

# # ----------------------------------------
# # TREND IMPACT MODELING (Simple Regression Model)
# # ----------------------------------------
# from sklearn.linear_model import LinearRegression

# def trend_impact_modeling(datasets, labels, trends):
#     st.subheader("Trend Impact Modeling on Sales")
#     pytrends = TrendReq(hl='en-US', tz=360)

#     impact_results = []

#     for trend in trends:
#         pytrends.build_payload([trend], cat=0, timeframe='today 3-m')
#         trend_df = pytrends.interest_over_time()

#         if not trend_df.empty:
#             trend_df = trend_df.drop(columns='isPartial').reset_index()
#             trend_df['Month'] = trend_df['date'].dt.month
#             trend_monthly = trend_df.groupby('Month')[trend].mean().reset_index()

#             for df, label in zip(datasets, labels):
#                 sales_monthly = df.groupby('Month')['Purchase Amount (USD)'].sum().reset_index()
#                 merged = pd.merge(trend_monthly, sales_monthly, on='Month', how='inner')
#                 if len(merged) > 1:
#                     X = merged[[trend]].values
#                     y = merged['Purchase Amount (USD)'].values
#                     model = LinearRegression()
#                     model.fit(X, y)
#                     coef = model.coef_[0]
#                     impact_results.append({"Dataset": label, "Trend": trend, "Impact Coefficient": coef})

#     if impact_results:
#         impact_df = pd.DataFrame(impact_results)
#         st.dataframe(impact_df)

# ----------------------------------------
# SALES BOOST FORECASTING BASED ON TRENDS
# ----------------------------------------

def sales_boost_forecast(df, selected_trends):
    st.subheader("Sales Boost Forecast Based on Selected Trends")
    pytrends = TrendReq(hl='en-US', tz=360)
    future_trend_score = 0

    for trend in selected_trends:
        pytrends.build_payload([trend], cat=0, timeframe='now 7-d')
        trend_df = pytrends.interest_over_time()
        if not trend_df.empty:
            score = trend_df[trend].mean()
            future_trend_score += score

    future_trend_score = future_trend_score / len(selected_trends) if selected_trends else 0

    sales_df = df[['Date Purchase', 'Purchase Amount (USD)']].rename(columns={
        'Date Purchase': 'ds',
        'Purchase Amount (USD)': 'y'
    })
    sales_df = sales_df.groupby('ds').sum().reset_index()

    model = Prophet()
    model.fit(sales_df)
    future = model.make_future_dataframe(periods=6, freq='M')
    forecast = model.predict(future)

    forecast['yhat_adjusted'] = forecast['yhat'] * (1 + future_trend_score / 100)

    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(forecast['ds'], forecast['yhat'], label='Base Forecast')
    ax.plot(forecast['ds'], forecast['yhat_adjusted'], label='Boosted Forecast', linestyle='--')
    plt.title("Sales Boost Forecast")
    plt.legend()
    st.pyplot(fig)

# ----------------------------------------
# RECOMMENDATION ENGINE
# ----------------------------------------

def recommendation_engine(trends, sentiments):
    st.subheader("Fashion Investment Recommendations")

    recommendations = []
    for trend in trends:
        sentiment = sentiments.get(trend, 0)
        if sentiment > 0.2:
            recommendations.append(f"✅ Invest more in {trend} category!")
        elif sentiment < -0.2:
            recommendations.append(f"⚠️ Caution advised for {trend} category.")

    if recommendations:
        for r in recommendations:
            st.write(r)
    else:
        st.info("No strong investment signals detected from sentiment analysis.")

# ----------------------------------------
# FUTURE FASHION INDEX
# ----------------------------------------

def future_fashion_index(trends):
    st.subheader("Future Fashion Index Tracker")

    if not trends:
        st.warning("No trends provided.")
        return

    index_scores = []

    for trend in trends:
        # Simulate a score between 30 and 80
        score = random.uniform(30, 80)
        index_scores.append(score)

    if index_scores:
        fashion_index = sum(index_scores) / len(index_scores)
        st.metric(label="Fashion Index Score", value=f"{fashion_index:.2f}")
    else:
        st.warning("Not enough trend data to calculate index.")


# ----------------------------------------
# INTEGRATION INTO MAIN STREAMLIT APP FLOW
# ----------------------------------------

st.title("From Hashtags to Hangars: Fashion Forecasting Dashboard")

uploaded_files = st.file_uploader("Upload one or more Fashion Sales CSVs", type=["csv"], accept_multiple_files=True)

if uploaded_files:
    datasets = load_multiple_csv(uploaded_files)
    dataset_labels = [file.name for file in uploaded_files]

    st.header("Forecast Manual Trends")
    trends_input = st.text_input("Enter multiple fashion trends separated by commas (e.g., barbiecore, grunge, Y2K)")
    if trends_input:
        trend_list = [trend.strip() for trend in trends_input.split(",")]
        forecast_multiple_trends(trend_list)
        sentiments = sentiment_analysis_on_trends(trend_list)
        st.header("Sentiment Analysis on Entered Trends")
        st.write(sentiments)

        st.header("Investment Recommendations Based on Trend Sentiment")
        recommendation_engine(trend_list, sentiments)

        st.header("Future Fashion Index")
        future_fashion_index(trend_list)

    if len(datasets) == 1:
        df = datasets[0]
        visualize_sales(df)
        model = train_sales_predictors(df)

        st.header("Future Sales Forecast")
        forecast_months = st.slider("Select number of months to forecast:", min_value=3, max_value=24, value=6)
        forecast_total_sales(df, periods=forecast_months)

        st.header("Predicted Top-Selling Items")
        predict_top_items(df, model)

        if trends_input:
            st.header("Sales Boost Forecast Based on Entered Trends")
            sales_boost_forecast(df, trend_list)

    else:
        st.header("Compare Datasets")
        compare_multiple_datasets(datasets, dataset_labels)

        individual_dataset_analytics(datasets, dataset_labels)

        st.header("Correlate Trends with Sales Across Datasets")
        trends_input_correlation = st.text_input("Enter trends to correlate (comma-separated):", key="trend_correlation")
        if trends_input_correlation:
            trend_list_correlation = [trend.strip() for trend in trends_input_correlation.split(",")]
            correlate_trends_with_datasets(datasets, dataset_labels, trend_list_correlation)
            detect_best_dataset(datasets, dataset_labels, trend_list_correlation)

            trend_impact_modeling(datasets, dataset_labels, trend_list_correlation)

        st.header("Master Combined Forecast Across All Datasets")
        master_combined_forecast(datasets, dataset_labels)

else:
    st.info("Upload one or more CSV files to get started!")

# # ----------------------------------------
# # UPDATED STREAMLIT APP Section (Full Analytics for Each Dataset)
# # ----------------------------------------

# st.title("From Hashtags to Hangars: Fashion Forecasting Dashboard")

# uploaded_files = st.file_uploader("Upload one or more Fashion Sales CSVs", type=["csv"], accept_multiple_files=True)

# if uploaded_files:
#     datasets = load_multiple_csv(uploaded_files)
#     dataset_labels = [file.name for file in uploaded_files]

#     st.header("Live Google Trends Scraper")
#     if st.button("Scrape Fashion Trends Now"):
#         scraped_trends = scrape_top_fashion_trends()
#         if scraped_trends:
#             st.session_state['scraped_trends'] = scraped_trends

#     if 'scraped_trends' in st.session_state:
#         selected_trends = st.multiselect("Select trends to forecast from scraped results:", st.session_state['scraped_trends'])
#         if selected_trends:
#             forecast_multiple_trends(selected_trends)

#     if len(datasets) == 1:
#         df = datasets[0]
#         visualize_sales(df)
#         model = train_sales_predictors(df)

#         st.header("Forecast Manual Trends")
#         trends_input = st.text_input("Enter multiple fashion trends separated by commas (e.g., barbiecore, grunge, Y2K)")
#         if trends_input:
#             trend_list = [trend.strip() for trend in trends_input.split(",")]
#             forecast_multiple_trends(trend_list)

#         st.header("Future Sales Forecast")
#         forecast_months = st.slider("Select number of months to forecast:", min_value=3, max_value=24, value=6)
#         sales_forecast = forecast_total_sales(df, periods=forecast_months)

#         st.header("Predicted Top-Selling Items")
#         predict_top_items(df, model)

#     else:
#         st.header("Compare Datasets")
#         compare_multiple_datasets(datasets, dataset_labels)

#         st.header("Individual Dataset Analytics")
#         for df, label in zip(datasets, dataset_labels):
#             st.subheader(f"Analytics for {label}")
#             visualize_sales(df)
#             model = train_sales_predictors(df)

#             st.subheader(f"Trend Forecasting for {label}")
#             trends_input = st.text_input(f"Enter trends to forecast for {label} (comma-separated):", key=f"forecast_{label}")
#             if trends_input:
#                 trend_list = [trend.strip() for trend in trends_input.split(",")]
#                 forecast_multiple_trends(trend_list)

#             st.subheader(f"Future Sales Forecast for {label}")
#             forecast_months = st.slider(f"Select forecast months for {label}:", min_value=3, max_value=24, value=6, key=f"months_{label}")
#             sales_forecast = forecast_total_sales(df, periods=forecast_months)

#             st.subheader(f"Predicted Top Items for {label}")
#             predict_top_items(df, model)

#         st.header("🔗 Correlate Trends with Sales Across Datasets")
#         trends_input = st.text_input("Enter trends to correlate (comma-separated):", key="trend_correlation")
#         if trends_input:
#             trend_list = [trend.strip() for trend in trends_input.split(",")]
#             correlate_trends_with_datasets(datasets, dataset_labels, trend_list)
#             detect_best_dataset(datasets, dataset_labels, trend_list)

#         st.header("Master Combined Forecast Across All Datasets")
#         master_combined_forecast(datasets, dataset_labels)
# else:
#     st.info("Upload one or more CSV files to get started!")
